In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from datetime import datetime, timedelta
import pdb
import os
import time
import tqdm

In [ ]:
'''
The srt files and txt files didn't have the same naming format.
The below code makes the names of files in both the directories consistent
by renaming the srt files to have the same name as the txt files.
'''

srt_mooc3_dir = "drive/MyDrive/mooc_transcript_data/transcripts_srt/mooc3_patterndiscovery_srt"
srt_mooc4_dir = "drive/MyDrive/mooc_transcript_data/transcripts_srt/mooc4_clusteranalysis_srt"
txt_mooc3_dir = "drive/MyDrive/mooc_transcript_data/transcripts_txt/mooc3_patterndiscovery_txt"
txt_mooc4_dir = "drive/MyDrive/mooc_transcript_data/transcripts_txt/mooc4_clusteranalysis_txt"


for file,file1 in zip(sorted(os.listdir(srt_mooc3_dir)),sorted(os.listdir(txt_mooc3_dir))):
  file_name = os.path.splitext(file1)[0]+".srt"
  old_file_path = os.path.join(srt_mooc3_dir,file)
  new_file_path = os.path.join(srt_mooc3_dir,file_name)
  os.rename(old_file_path, new_file_path)

for file,file1 in zip(sorted(os.listdir(srt_mooc4_dir)),sorted(os.listdir(txt_mooc4_dir))):
  file_name = os.path.splitext(file1)[0]+".srt"
  old_file_path = os.path.join(srt_mooc4_dir,file)
  new_file_path = os.path.join(srt_mooc4_dir,file_name)
  os.rename(old_file_path, new_file_path)


In [ ]:
'''
creates a dictionary 
key : path to srt file
value: (path_to_txt_file, path_to_mintxt_file)
'''

srt_directory_path = "drive/MyDrive/mooc_transcript_data/transcripts_srt/"
txt_directory_path = "drive/MyDrive/mooc_transcript_data/transcripts_txt/"
mintxt_directory_path = "drive/MyDrive/mooc_transcript_data/transcripts_mintxt/"
if not os.path.exists(mintxt_directory_path):
    os.makedirs(mintxt_directory_path)
srt_dir_path_dict = dict()

for srt_dir in os.listdir(srt_directory_path):
  if len(srt_dir.split("_"))==3 and srt_dir.split("_")[2] != "webvtt":
    
    srt_path = os.path.join(srt_directory_path, srt_dir) #srt directory paths 
    txt_dir = srt_dir[0:len(srt_dir)-3]+"txt" 
    mintxt_dir = srt_dir[0:len(srt_dir)-3]+"mintxt"
    txt_path = os.path.join(txt_directory_path, txt_dir) #txt directory paths
    mintxt_path = os.path.join(mintxt_directory_path, mintxt_dir)
    if not os.path.exists(mintxt_path):
      os.makedirs(mintxt_path) #mintxt directory paths

    if os.path.isdir(srt_path):
      files = os.listdir(srt_path) #srt file path
      for file in files:
        file_extension = os.path.splitext(file)[1]
        if file_extension == ".srt":
          src_file_path = os.path.join(srt_path,file)
          txt_file_name = os.path.splitext(file)[0]+".txt"
          mintxt_file_path = os.path.join(mintxt_path,txt_file_name)
          with open(mintxt_file_path,'w') as file:
            pass
          txt_file_path = os.path.join(txt_path,txt_file_name) #txt file path
          srt_dir_path_dict[src_file_path]= (txt_file_path,mintxt_file_path)


# for k,v in srt_dir_path_dict.items():
#   print(k,v)


In [ ]:
def seg_txt(srt_path):
  '''
  input: path to the srt file
  ouput: list of the last line of each 1 minute segment
  Function: uses the timestamps in .srt files to create 1-minute segments of transcripts
  '''
  with open(srt_path,"r") as f:
      srt_content = f.read()

  srt_list = srt_content.split("\n\n") #split the srt file into time segments in the file
  srt_lines = srt_content.split("\n") #iterating every line
  begin_list, end_list = list(), list()

  srt_list_timestamp = list()
  for line in srt_lines:
    if "-->" in line:
      begin = line.split("-->")[0].split(",")[0]
      end  = line.split("-->")[1].split(",")[0].lstrip()
      begin = datetime.strptime(begin, "%H:%M:%S")
      end = datetime.strptime(end, "%H:%M:%S")
      begin_list.append(begin)
      end_list.append(end)
  
  i , j = 0, 0
  flag = 0
  seg_txt = list()
  while i < len(begin_list):
    for j in range(i,len(end_list)):
      diff = end_list[j] - begin_list[i]
      if diff.seconds >= 60:
        seg_txt.append(srt_list[j].split("\n")[-1]) #return only last line of the segment
        i = j+1
        flag = 1
        break
    if flag == 1:
      flag = 0
    else:
      i = i + 1
  
  return seg_txt

In [ ]:
def delim_add_txt(txt_path,mintxt_path,seg_txt):
  '''
  input: path_to_txt_file, path_to_mintxt_file, list of 1 minute segment txts
  output: None 
  Function: Delimits the transcripts at 1-minute segments
  '''
  delimiter = "||"
  seg_txt = {k:False for k in seg_txt} #false implies that the segment hasn't been parsed 
  # read the file and split into lines
  with open(txt_path, "r") as file:
      lines = file.read().splitlines()

  # loop through each line and check if it matches any of the lines to find
  for i, line in enumerate(lines):
      for delim_line in seg_txt:
        if seg_txt[delim_line] == False and delim_line in line:
            b_idx = line.find(delim_line) #locate exactly where the minute segment text is present
            e_idx = b_idx + len(delim_line)
            lines[i] = line[0:e_idx] + delimiter + line[e_idx:]
            seg_txt[delim_line] = True
  

  # write the modified lines to the minute segment text file
  with open(mintxt_path, "w") as file:
      file.write("\n".join(lines))


In [ ]:
for k,v in tqdm.tqdm(srt_dir_path_dict.items()):
  min_seg_txt = seg_txt(k)
  delim_add_txt(v[0],v[1],min_seg_txt)


100%|██████████| 181/181 [00:02<00:00, 75.09it/s]
